# ***Distribución BAÑOS***

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "alfa" in nombre and "pachuca" in nombre

    #if nombre.startswith("df_") and "tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_pachuca',
 'df_alfa_agosto_2024_pachuca',
 'df_alfa_febrero_2024_pachuca',
 'df_alfa_julio_2024_pachuca',
 'df_alfa_junio_2024_pachuca',
 'df_alfa_marzo_2024_pachuca',
 'df_alfa_mayo_2024_pachuca',
 'df_alfa_q_feb_2023_pachuca',
 'df_alfa_q_jul_2023_pachuca',
 'df_alfa_q_jun_2023_pachuca',
 'df_alfa_q_mar_2023_pachuca',
 'df_alfa_q_nov_2022_pachuca',
 'df_alfa_q_oct_2022_pachuca',
 'df_alfa_q_sep_2023_pachuca']

In [4]:
df_alfa_junio_2024_pachuca.columns

Index(['id', 'Categoria', 'Precio', 'propiedad', 'metros_total',
       'metros_construido', 'precio_m2_terreno', 'precio_m2_construido',
       'tiempo_de_publicacion', 'Estado', 'Tipo', 'Estacionamientos',
       'Recamaras', 'Banos', 'Medio_banos', 'Banos_Total', 'Seguridad',
       'Colonia', 'CP', 'url'],
      dtype='object')

In [5]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'Categoria':'categoria','Banos_Total':'baño_total','id_jul_23':'id','m2_contruido': 'm2_construido','m_construido': 'm2_construido','Metros_construido':'m2_construido','segmento':'categoria','Category':'categoria','m_total':'m2_total','m_construido':'m2_construido',}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [6]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id','categoria','baño_total']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

## Total de baños

In [7]:
def procesar_dataframes(dataframes_list, nombres_df_filtrados):
    for i, df in enumerate(dataframes_list):
        # Crear una nueva columna con la primera letra de la categoría
        df['segmento'] = df['categoria'].str[0]
        df.drop(columns=['categoria'], inplace=True)
        df['baño_total'] = df['baño_total'].clip(upper=8) # Limitar los valores de 'baño_total' a un máximo de 8
        #df['baño_total'] = pd.cut(df['baño_total'], bins=bins, labels=labels, right=False)

        # Agrupar y contar los datos
        tabla = df.groupby(['segmento', 'baño_total']).size().unstack(fill_value=0)

        # Imprimir el nombre del DataFrame y la tabla resultante
        print(f"Nombre del DataFrame: {nombres_df_filtrados[i]}")
        print(tabla)

# Procesar los DataFrames de la lista
procesar_dataframes(dataframes_list, nombres_df_filtrados)

Nombre del DataFrame: df_alfa_abril_2024_pachuca
baño_total    1   2   3   4   5  6  7
segmento                             
A            59  32  13  18  11  0  0
B            92  41  32  24   8  0  0
C           122  59  48  15   2  1  0
D           179  96  56   4   1  0  0
E           137  60  13   1   0  0  0
L             1   0   0   0   0  0  0
S           119  52  44  24  15  2  1
Nombre del DataFrame: df_alfa_agosto_2024_pachuca
baño_total  0.0  1.0  1.5  2.0  2.5  3.0  3.5  4.0  4.5  5.0  5.5  6.0  6.5  \
segmento                                                                      
A             0    2    0   34   14   17   35   12    7    7    1    0    0   
B             0    2    3   38   53   26   39   18    7    4    1    0    0   
C             0   10   18   41   78   37   19   15    2    1    0    0    0   
D             1   52   51   65   40   19    4    1    0    3    0    0    0   
E             0   73   19   31   31    9   10    2    2    2    0    1    0   
L     

C:\Users\yoe11\AppData\Local\Temp\ipykernel_23932\705114878.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['segmento'] = df['categoria'].str[0]
C:\Users\yoe11\AppData\Local\Temp\ipykernel_23932\705114878.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['categoria'], inplace=True)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_23932\705114878.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

In [8]:
# Crear una nueva función para sumar las tablas de todos los DataFrames
def sumar_tablas(dataframes_list):
    # Inicializar una tabla vacía para almacenar la suma
    tabla_suma = None
    
    # Iterar sobre cada DataFrame en la lista
    for df in dataframes_list:
        # Agrupar y contar los datos en el DataFrame actual
        tabla_actual = df.groupby(['segmento', 'baño_total']).size().unstack(fill_value=0)
        
        # Sumar la tabla actual al acumulador (tabla_suma)
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    # Imprimir la tabla resultante de la suma
    print("Tabla general de frecuencia:")
    print(tabla_suma)

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
sumar_tablas(dataframes_list)

Tabla general de frecuencia:
baño_total   0.0  0.5     1.0    1.5     2.0    2.5     3.0   3.5    4.0  \
segmento                                                                   
A           11.0  0.0   350.0    1.0   494.0   29.0   487.0  76.0  553.0   
B           16.0  0.0   558.0   14.0   813.0  107.0  1068.0  75.0  768.0   
C           13.0  0.0   770.0   38.0   991.0  174.0  1198.0  35.0  270.0   
D           20.0  0.0  1617.0  120.0  1441.0   96.0   754.0  12.0   73.0   
E            6.0  1.0  1204.0   52.0   434.0   85.0   166.0  26.0   41.0   
L            2.0  0.0    40.0    0.0    58.0    2.0    20.0   5.0   59.0   
S           25.0  1.0   671.0    1.0   653.0   57.0   788.0  98.0  912.0   

baño_total   4.5    5.0   5.5    6.0  6.5   7.0   8.0  
segmento                                               
A           17.0  309.0   3.0   78.0  0.0  20.0   6.0  
B           15.0  230.0   2.0   27.0  0.0   9.0   3.0  
C            4.0   40.0   0.0    8.0  0.0   4.0   1.0  
D     

In [9]:
import os
import plotly.graph_objects as go
import plotly.io as pio

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#F79CB9']

def sumar_tablas(dataframes_list):
    tabla_suma = None
    for df in dataframes_list:
        tabla_actual = df.groupby(['segmento', 'baño_total']).size().unstack(fill_value=0)
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    print("Tabla general de frecuencia:")
    print(tabla_suma)
    
    fig = graficar_tabla(tabla_suma)
    guardar_grafico_como_html(fig, 'g_bar_baños', carpeta='assets/graficas') # Para guardar la gráfica como archivo HTML
    return fig

def graficar_tabla(tabla):
    segmentos = tabla.index
    banos = tabla.columns
    
    fig = go.Figure()
    
    for i, segmento in enumerate(segmentos):
        color = colores[i % len(colores)]
        fig.add_trace(go.Bar(
            x=banos,
            y=tabla.loc[segmento],
            name=segmento,
            marker_color=color
        ))
    
    fig.update_layout(
        xaxis_title="Número de baños",        
        yaxis=dict(
            title='Número de casas',
            gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
            gridwidth=1,
        ),
        legend_title="Segmento",
        margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    )    
    
    fig.show()    
    return fig

# Guardar la gráfica como archivo HTML en la carpeta especificada
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
fig = sumar_tablas(dataframes_list)

Tabla general de frecuencia:
baño_total   0.0  0.5     1.0    1.5     2.0    2.5     3.0   3.5    4.0  \
segmento                                                                   
A           11.0  0.0   350.0    1.0   494.0   29.0   487.0  76.0  553.0   
B           16.0  0.0   558.0   14.0   813.0  107.0  1068.0  75.0  768.0   
C           13.0  0.0   770.0   38.0   991.0  174.0  1198.0  35.0  270.0   
D           20.0  0.0  1617.0  120.0  1441.0   96.0   754.0  12.0   73.0   
E            6.0  1.0  1204.0   52.0   434.0   85.0   166.0  26.0   41.0   
L            2.0  0.0    40.0    0.0    58.0    2.0    20.0   5.0   59.0   
S           25.0  1.0   671.0    1.0   653.0   57.0   788.0  98.0  912.0   

baño_total   4.5    5.0   5.5    6.0  6.5   7.0   8.0  
segmento                                               
A           17.0  309.0   3.0   78.0  0.0  20.0   6.0  
B           15.0  230.0   2.0   27.0  0.0   9.0   3.0  
C            4.0   40.0   0.0    8.0  0.0   4.0   1.0  
D     

In [10]:
def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
        tabla_actual = df.groupby('baño_total').size()
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    return tabla_suma

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
tabla_suma = sumar_tablas(dataframes_list)
print("Tabla general total de registros:")
print(tabla_suma)


Tabla general total de registros:
baño_total
0.0     108.0
0.5       2.0
1.0    5666.0
1.5     236.0
2.0    5140.0
2.5     637.0
3.0    4638.0
3.5     383.0
4.0    2758.0
4.5     156.0
5.0    1308.0
5.5      42.0
6.0     344.0
6.5       4.0
7.0      84.0
8.0      52.0
dtype: float64


In [11]:
# Calcular el total de registros
total_registros = tabla_suma.sum()
# Calcular el porcentaje que representa cada número de baños único del total
porcentajes = (tabla_suma / total_registros) * 100

# Imprimir los porcentajes
print("Porcentaje de cada número de baños único en relación con el total de registros:", total_registros)
print(porcentajes)


Porcentaje de cada número de baños único en relación con el total de registros: 21558.0
baño_total
0.0     0.500974
0.5     0.009277
1.0    26.282587
1.5     1.094721
2.0    23.842657
2.5     2.954820
3.0    21.514055
3.5     1.776603
4.0    12.793395
4.5     0.723629
5.0     6.067353
5.5     0.194823
6.0     1.595695
6.5     0.018555
7.0     0.389647
8.0     0.241210
dtype: float64


In [12]:
import os
import plotly.graph_objects as go
import plotly.io as pio

# Lista de colores
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#F79CB9','#E50CB6','#4225CF']

# Función para sumar tablas
def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
        tabla_actual = df.groupby('baño_total').size()
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    return tabla_suma

# Función para calcular porcentajes
def calcular_porcentajes(tabla):
    # Calcular los porcentajes
    tabla_porcentajes = tabla / tabla.sum() * 100
    return tabla_porcentajes

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
tabla_suma = sumar_tablas(dataframes_list)
porcentajes = calcular_porcentajes(tabla_suma) # Calcular porcentajes

# Crear el gráfico de pastel con colores especificados
figURE = go.Figure(data=[go.Pie(labels=porcentajes.index, values=porcentajes, sort=False, marker=dict(colors=colores))])
figURE.update_layout( title_x=0.5, title_font_size=20,margin=dict(l=10, r=10, t=10, b=10),  )
figURE.update_traces(rotation=45) # Ángulo de inicio de las porciones del gráfico

# Exportar gráfica como archivo HTML y mostrar la figura
def guardar_grafico_como_html(figURE, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(figURE, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(figURE, 'g_pie_baños', carpeta='assets/graficas')

# Mostrar la figura
figURE.show()